# m3u8

HLS形式動画のプレイリストのファイル形式です。


## HLS 

HLS (HTTP Live Streaming) は動画のストリーミング配信のプロトコルです。
HLSで動画を配信するときに関連するファイルは3つあります。

| 種類 | 拡張子 | 内容 |
|:-----------|------------:|:------------:|
| tsファイル | ts | 実際の動画データ |
| プレイリスト | m3u8 | 動画データをどの順番に再生すれば良いかなどの動画のメタデータを保持する |
| 鍵 | 特になし | tsファイルを暗号化している場合、複合化する鍵のバイナリデータ |


## m3u8ファイル

m3u8ファイルは動画のメタデータなどを保持するためのファイルです。
HLSの動画は分割されていますのでどこのURLの動画をどの順番に再生するのかという情報などを持っています。

m3u8ファイルはこんな感じ。

```
#EXTM3U
#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=200000
movie_1.m3u8
#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=311111
movie_2.m3u8
#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=484444
movie_3.m3u8
#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=737777
movie_4.m3u8
```

# とりあえず使って覚える


https://github.com/globocom/m3u8


In [65]:
import m3u8
playlist = m3u8.M3U8()
print(playlist.dumps())

#EXTM3U



dumpsとかdumpとかloadsとかloadとか見慣れたmethodが用意されている。

## 動画追加してみる

動画情報はsegmentとよばれているやつっぽい。

In [66]:
segment = m3u8.Segment()

TypeError: __init__() missing 2 required positional arguments: 'uri' and 'base_uri'

ううっ、uriとbase_urlが必要らしい。

In [67]:
segment = m3u8.Segment(base_uri='http://example.com', uri='1.ts')

良さそう

In [68]:
playlist.segments.append(segment)

In [69]:
print(playlist.dumps())

TypeError: a float is required

number? 何かが必要らしい。 titleとか表示されているのでtitle入れてみる。

In [70]:
segment.title = 'aaa'
print(playlist.dumps())

TypeError: a float is required

duration入れてみる。 (durationの単位は秒)

In [71]:
segment.duration = 1
print(playlist.dumps())

#EXTM3U
#EXTINF:1,"aaa"
1.ts


### yay (๑•̀ㅂ•́)و✧

....でもbase_uriって値を入れた割には使われていないなあ。なんでだ?

### segment増やしてみる

In [72]:
playlist.segments.append(m3u8.Segment(base_uri='http://example.com/', uri='2.ts', duration=2))
playlist.segments.append(m3u8.Segment(base_uri='http://example.com/', uri='3.ts', duration=3))
playlist.segments.append(m3u8.Segment(base_uri='http://example.com/', uri='4.ts', duration=4))
print(playlist.dumps())

#EXTM3U
#EXTINF:1,"aaa"
1.ts
#EXTINF:2,
2.ts
#EXTINF:3,
3.ts
#EXTINF:4,
4.ts


##  playlistをloadする

今度は逆!! playlistの文字列からM3U8 objectを生成します。

In [75]:
playlist_str = playlist.dumps()
print(playlist_str)

#EXTM3U
#EXTINF:1,"aaa"
1.ts
#EXTINF:2,
2.ts
#EXTINF:3,
3.ts
#EXTINF:4,
4.ts


In [78]:
playlist_2= m3u8.M3U8(playlist_str)
print(playlist_2.dumps())

#EXTM3U
#EXTINF:1,"aaa"
1.ts
#EXTINF:2,
2.ts
#EXTINF:3,
3.ts
#EXTINF:4,
4.ts


コンストラクタに渡せば良い模様

## 鍵つけてみる

HLSはコンテンツ保護のためtsファイルを暗号化し、その複合化の鍵をプレイリストに持たせることができます。
※ 暗号化はopensslコマンドとかでやれます!!

今回はplaylistに複合鍵のURIをm3u8 object経由で出力してみます。

ここを参考にする https://github.com/globocom/m3u8#encryption-key 


In [88]:
key = m3u8.model.Key('AES-256', uri='keyfile', base_uri='http://example.com', iv='000000000000000')

In [89]:
playlist.key = key
print(playlist.dumps())

#EXTM3U
#EXT-X-KEY:METHOD=AES-256,URI="keyfile",IV=000000000000000
#EXTINF:1,"aaa"
1.ts
#EXTINF:2,
2.ts
#EXTINF:3,
3.ts
#EXTINF:4,
4.ts


お!! できた。それにしてもbase_uriが謎。別にどこかに出力されるわけではないのに必須パラメータ.....

In [90]:
playlist.key = m3u8.model.Key('AES-256', uri='http://example.com/keyfile', base_uri='http://example.com', iv='000000000000000')
print(playlist.dumps())

#EXTM3U
#EXT-X-KEY:METHOD=AES-256,URI="http://example.com/keyfile",IV=000000000000000
#EXTINF:1,"aaa"
1.ts
#EXTINF:2,
2.ts
#EXTINF:3,
3.ts
#EXTINF:4,
4.ts


うむー。。。。わからん。

In [91]:
playlist.key = m3u8.model.Key('AES-256', base_path='keyfile', base_uri='http://example.com', iv='000000000000000')
print(playlist.dumps())

TypeError: __init__() got an unexpected keyword argument 'base_path'

うむーうむー。。。わからん。
base_uriとbase_pathとuriの扱いがわからんん。

- とりあえずuriは直接URIを記載すればその通りになる。
- base_uriはBasePathMixin.absolute_uriがあるのでそちらを使うと使えそう。 BasePathMixinはKeyとかSegmentとかSegmentListとかPlaylistとか要所で継承している。
- base_pathは....

In [95]:
m3u = m3u8.M3U8()
m3u.segments.append(m3u8.Segment(base_uri='segment_base_uri', uri='segment_uri', duration=2))
m3u.key = m3u8.model.Key('AES-256', base_uri='key_base_uri', uri='key_uri' , iv='000000000000000')
print(m3u.dumps())

#EXTM3U
#EXT-X-KEY:METHOD=AES-256,URI="key_uri",IV=000000000000000
#EXTINF:2,
segment_uri


In [102]:
m3u = m3u8.M3U8()
m3u.segments.append(m3u8.Segment(base_uri='segment_base_uri', uri='segment_uri', duration=2))
m3u.key = m3u8.model.Key('AES-256', base_uri='key_base_uri', uri='key_uri' , iv='000000000000000')
media = m3u8.Media()
m3u8.Playlist(uri='playlist_uri', base_uri='playlist_base_uri', media=media, stream_info=None)


AttributeError: 'NoneType' object has no attribute 'get'